In [1]:


# On an undirected graph, the adjacency matrix is symmetric.



In [2]:
exec(open("./funcs/tool_funcs.py").read())



In [3]:
def dijkstra_show_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  # Only consider edges with weight > 0
                distance = current_distance + weight

                # Update distance if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [4]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [5]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [6]:
def cal_candidate_node_list_symmetric(i, j, shortest_path_vectors, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if shortest_path_vectors[0][j] < shortest_path_vectors[0][t] + shortest_path_vectors[1][t]:
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        

In [7]:
 
def cal_SPD(X_distance_matrix, SPD_path):
    temp1= SPD_path[:-1]
    temp2= SPD_path[1:]
    llll = [X_distance_matrix[temp1[i],temp2[i]] for i in range(len(temp1))]
    return np.sum(llll) 

In [8]:
 
def check_shortest_path_SPD(i,j, X_distance_matrix, SPD, path):
    
    
    temp1= path[:-1]
    temp2= path[1:]
    llll = [X_distance_matrix[temp1[i],temp2[i]] for i in range(len(temp1))]
    ooo = np.sum(llll)
    return ooo == SPD

In [9]:
def check_if_temp_path_already_in(all_paths_list, temp_path):
    aa = set(temp_path)
    for bb in all_paths_list:
        bb = set(bb)
        if aa == bb:
            return True
    return False

def cal_all_paths_cold_start_undirected_graph(i, j, shortest_path_vectors, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list_symmetric(i, j, shortest_path_vectors, X_distance_matrix)

    print("candidate_node_list:")
    print(candidate_node_list)
    
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path(small_matrix, 0, len(candidate_node_list) - 1)

    all_paths_list = []
    all_paths_list.append(path)

    K = len(candidate_node_list)
    # temp_list = [q for q in range(K) if q not in path]
    
    temp_list = [q for q in range(1, K - 1)]

    for m in temp_list:
        temp_path1 = dijkstra_show_path(small_matrix, 0, m)
        temp_path2 = dijkstra_show_path(small_matrix, m, K - 1)
        temp_path =  temp_path1[:-1] + temp_path2
        if not check_if_temp_path_already_in(all_paths_list, temp_path):
            all_paths_list.append(temp_path)

    all_paths_list = [translate_path(ppp, candidate_node_list) for ppp in all_paths_list]    

    return all_paths_list

In [10]:
 

def dijkstra_one_to_all_simple(distance_matrix, src):
    
    N = len(distance_matrix)
    dist = [np.inf] * N
    dist[src] = 0
    visited = [False] * N

    # Priority queue: (distance, node)
    pq = [(0, src)]
    
    while pq:
        current_dist, u = heapq.heappop(pq)

        if visited[u]:
            continue
        visited[u] = True

        # Update distances for neighbors
        for v in range(N):
            if not visited[v] and distance_matrix[u][v] > 0:
                new_dist = current_dist + distance_matrix[u][v]
                if new_dist < dist[v]:
                    dist[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))

    return np.array(dist)



In [11]:
def make_dis_matrix_symmetric(dist_matrix):
    n = len(dist_matrix)    
 
    for i in range(n):
        for j in range(n):
            if i < j:
                dist_matrix[j,i] = dist_matrix[i,j]
 
    return dist_matrix

In [12]:
# N = 1000

# X_distance_matrix = create_distance_matrix(N)

X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")



In [13]:
X_distance_matrix = make_dis_matrix_symmetric(X_distance_matrix)


In [14]:
X_distance_matrix

array([[  0., 952., 405., ..., 208., 152., 808.],
       [952.,   0., 557., ..., 912., 737.,  17.],
       [405., 557.,   0., ..., 932., 459., 276.],
       ...,
       [208., 912., 932., ...,   0., 984., 276.],
       [152., 737., 459., ..., 984.,   0., 241.],
       [808.,  17., 276., ..., 276., 241.,   0.]])

In [15]:

N = len(X_distance_matrix)


In [16]:
i = 22
j = 93
 
SPD_path = dijkstra_show_path(X_distance_matrix, i, j)

 
    

In [17]:
# SPD stands for shortest path distance.

SPD = cal_SPD(X_distance_matrix, SPD_path)
 

In [18]:
check_shortest_path_SPD(i,j, X_distance_matrix, SPD, SPD_path)



True

In [19]:
t1 = dijkstra_one_to_all_simple(X_distance_matrix, i)
t2 = dijkstra_one_to_all_simple(X_distance_matrix, j)

shortest_path_vectors = [t1, t2]


In [20]:
all_paths_list = cal_all_paths_cold_start_undirected_graph(i, j, shortest_path_vectors, X_distance_matrix)

print("shortest_path: ")
   
print(SPD_path)


candidate_node_list:
[22, 45, 61, 82, 93]
shortest_path: 
[22, 45, 82, 61, 93]


In [21]:
# There is only one shortest path between node 22 and 93.

for path_i in all_paths_list:
    aaa = check_shortest_path_SPD(i,j, X_distance_matrix, SPD, path_i)
    print(path_i, aaa)



[22, 45, 82, 61, 93] True
